In [1]:
import langchain
print(langchain.__version__)

0.3.4


In [ ]:
from typing import Optional
import os
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, model_validator  # model_validator 임포트

llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=300,
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY")
)

# Pydantic
class Categorized(BaseModel):
    common_conversation: bool = Field(
        description="""
            사용자가 일반적인 대화를 원하면 True, 그렇지 않으면 False를 대답해줘
        """
    )
    SQLQuery: bool = Field(
        description="""
            사용자가 쿼리문을 요청하거나, 데이터에 대한 정보를 원하면 True, 그렇지 않으면 False를 대답해줘.
        """
    )

    @model_validator(mode="after")
    def check_only_one_true(cls, values):
        if values.common_conversation == values.SQLQuery:  # 둘 다 True 또는 둘 다 False인 경우 에러 발생
            raise ValueError("common_conversation과 SQLQuery 중 하나만 True여야 합니다.")
        return values

structured_llm = llm.with_structured_output(Categorized)

structured_llm.invoke("Tell me a joke about cats")


Categorized(common_conversation=True, SQLQuery=False)

In [7]:
# 의사 분류기 테스트
result = structured_llm.invoke("저번주에 로그인한 유저수")
result

Categorized(common_conversation=False, SQLQuery=True)

In [8]:
result = structured_llm.invoke("Tell me a joke about cats")
result

Categorized(common_conversation=True, SQLQuery=False)

In [9]:
result = structured_llm.invoke("이번주에 로그인한 유저수 알려줘. 저번주에 로그인한 유저가 마음에 안들어. ")
result

Categorized(common_conversation=False, SQLQuery=True)

In [10]:
result = structured_llm.invoke("이 대화는 일상적인 대화와 동시에 text-to-sql을 원하는 대화야")
result

ValidationError: 1 validation error for Categorized
  Value error, common_conversation과 SQLQuery 중 하나만 True여야 합니다. [type=value_error, input_value={'common_conversation': True, 'SQLQuery': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error